In [1]:
import warnings
import tensorflow as tf
warnings.filterwarnings("ignore")
tf.get_logger().setLevel('ERROR')

import numpy as np
import pandas as pd
import deepchem as dc
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import torch
from torch import nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, global_max_pool
from torch_geometric.data import Batch, Data
from torch_geometric.loader import DataLoader
from sklearn import metrics

Skipped loading modules with pytorch-geometric dependency, missing a dependency. No module named 'dgl'
Skipped loading some Jax models, missing a dependency. No module named 'jax'


In [2]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = GCNConv(30, 256)
        self.conv2 = GCNConv(256, 256)
        self.conv3 = GCNConv(256, 256)
        self.conv4 = GCNConv(256, 256)
        self.fc1 = nn.Linear(256, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 1)
        self.dropout1 = nn.Dropout(p=0.15)
        self.dropout2 = nn.Dropout(p=0.3)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.dropout1(x)
        x = self.conv2(x, edge_index)
        x = F.relu(x)
        x = self.conv3(x, edge_index)
        x = F.relu(x)
        x = self.conv4(x, edge_index)
        x = F.relu(x)
        x = global_max_pool(x, data.batch)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        x = F.relu(x)
        x = self.fc3(x)
        return x

In [3]:
def custom_collate(batch):
    data_list, target_list = zip(*batch)
    batch_data = Batch.from_data_list(data_list)
    batch_target = torch.stack(target_list)
    return batch_data, batch_target

In [4]:
def calculate_statistics(group):
    r2_test = group['r2_test']
    r2_test_dict = {f'run{i}': r2_test_val for i, r2_test_val in enumerate(r2_test)}
    return pd.Series({
        **r2_test_dict, 
        'r2_test_mean': np.mean(r2_test),
        'r2_test_max': np.max(r2_test),
        'r2_test_min': np.min(r2_test),
        'r2_test_std': np.std(r2_test, ddof=0),
    })

def calculate_statistics2(group):
    rmse_test = group['rmse_test']
    rmse_test_dict = {f'run{i}': rmse_test_val for i, rmse_test_val in enumerate(rmse_test)}
    return pd.Series({
        **rmse_test_dict, 
        'rmse_test_mean': np.mean(rmse_test),
        'rmse_test_max': np.max(rmse_test),
        'rmse_test_min': np.min(rmse_test),
        'rmse_test_std': np.std(rmse_test, ddof=0),
    })

In [5]:
torch.manual_seed(0)

first_epochs = 50
second_epochs = 200
second_lr = 9e-3
second_wd = 0

results_r2 = []
results_rmse = []
for random_state in range(10):
    torch.manual_seed(0)
    scaler = StandardScaler()
    
    for dataset in ["abcgg", "aatsc3d", "atsc3d", "kappa2", "peoevsa6", "bertzct", "ggi10", "vsaestate3",
                    "atsc4i", "bcutp1l", "kappa3", "estatevsa3", "kier3", "aats8p", "kier2", "frnh0"]:
        torch.manual_seed(0)
        
        for t in ["Yield_CO_l"]:
            torch.manual_seed(0)
            df = pd.read_csv('data_Real/data_real.csv')
            smiles = df["SMILES"]
            featurizer = dc.feat.MolGraphConvFeaturizer(use_edges=True)
            X = featurizer.featurize(smiles)
            
            y = df[t]
            data_train, data_test, target_train, target_test = train_test_split(X, y, test_size=0.5, random_state=random_state)

            target_train = scaler.fit_transform(target_train.values.reshape(-1, 1)).flatten()
            target_test = scaler.transform(target_test.values.reshape(-1, 1)).flatten()
            
            target_train = torch.tensor(target_train, dtype=torch.float32)
            target_test = torch.tensor(target_test, dtype=torch.float32)

            data_train_list = []
            for graph_data in data_train:
                node_features = torch.tensor(graph_data.node_features, dtype=torch.float32)
                edge_index = torch.tensor(graph_data.edge_index, dtype=torch.long)
                edge_features = torch.tensor(graph_data.edge_features, dtype=torch.float32)
                data = Data(x=node_features, edge_index=edge_index, edge_attr=edge_features)
                data_train_list.append(data)

            data_test_list = []
            for graph_data in data_test:
                node_features = torch.tensor(graph_data.node_features, dtype=torch.float32)
                edge_index = torch.tensor(graph_data.edge_index, dtype=torch.long)
                edge_features = torch.tensor(graph_data.edge_features, dtype=torch.float32)
                data = Data(x=node_features, edge_index=edge_index, edge_attr=edge_features)
                data_test_list.append(data)

            train_loader = DataLoader(list(zip(data_train_list, target_train)), batch_size=len(data_train_list), collate_fn=custom_collate)
            test_loader = DataLoader(list(zip(data_test_list, target_test)), batch_size=len(data_test_list), collate_fn=custom_collate)

            model = Net()
            model.load_state_dict(torch.load(f'data_AI2+Human/model_{dataset}_sc.pth'))
            model.fc3 = nn.Linear(128, 1)
        
            for param in model.parameters():
                param.requires_grad = False
        
            model.train()
            optimizer = torch.optim.Adam(model.parameters())
            criterion = nn.MSELoss()
            
            device = torch.device('cpu')
            model.to(device)

            for epoch in range(first_epochs):
                for data, target in train_loader:
                    data = data.to(device)
                    target = target.to(device)
                    with torch.no_grad():
                        out = model(data)
                        loss = criterion(out, target.view(-1, 1))

            for param in model.fc1.parameters():
                param.requires_grad = True
            for param in model.fc2.parameters():
                param.requires_grad = True
            for param in model.fc3.parameters():
                param.requires_grad = True

            optimizer = torch.optim.Adam(model.parameters(), lr=second_lr, weight_decay=second_wd)

            for epoch in range(second_epochs):
                for data, target in train_loader:
                    data = data.to(device)
                    target = target.to(device)
                    optimizer.zero_grad()
                    out = model(data)
                    loss = criterion(out, target.view(-1, 1))
                    loss.backward()
                    optimizer.step()

            model.eval()
            pred_train = []
            for data, target in train_loader:
                data = data.to(device)
                with torch.no_grad():
                    out = model(data)
                pred_train.append(out.cpu().numpy())
            pred_train = np.concatenate(pred_train)

            pred_test = []
            for data, target in test_loader:
                data = data.to(device)
                with torch.no_grad():
                    out = model(data)
                pred_test.append(out.cpu().numpy())
            pred_test = np.concatenate(pred_test)

            pred_train = scaler.inverse_transform(pred_train)
            pred_test = scaler.inverse_transform(pred_test)
            target_train = scaler.inverse_transform(target_train.numpy().reshape(-1, 1)).flatten()
            target_test = scaler.inverse_transform(target_test.numpy().reshape(-1, 1)).flatten()

            r2_test_score = metrics.r2_score(target_test, pred_test)
            rmse_test_score = metrics.root_mean_squared_error(target_test, pred_test)
            results_r2.append({'source': dataset, 'target': t, 'r2_test': r2_test_score})
            results_rmse.append({'source': dataset, 'target': t, 'rmse_test': rmse_test_score})

results_df = pd.DataFrame(results_r2)
gen_results = results_df.groupby(['source', 'target']).apply(calculate_statistics).reset_index()
results_df2 = pd.DataFrame(results_rmse)
gen_results2 = results_df2.groupby(['source', 'target']).apply(calculate_statistics2).reset_index()

In [6]:
gen_results.T.to_csv('result/result_yield_l_r2.csv', header=False)
gen_results.T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
source,aats8p,aatsc3d,abcgg,atsc3d,atsc4i,bcutp1l,bertzct,estatevsa3,frnh0,ggi10,kappa2,kappa3,kier2,kier3,peoevsa6,vsaestate3
target,Yield_CO_l,Yield_CO_l,Yield_CO_l,Yield_CO_l,Yield_CO_l,Yield_CO_l,Yield_CO_l,Yield_CO_l,Yield_CO_l,Yield_CO_l,Yield_CO_l,Yield_CO_l,Yield_CO_l,Yield_CO_l,Yield_CO_l,Yield_CO_l
run0,0.64502,0.613496,0.830758,0.69149,0.681436,0.523976,0.820812,0.605799,0.686299,0.716561,0.72319,0.765868,0.788164,0.79839,0.765529,0.702792
run1,0.710586,0.361782,0.708678,0.373749,0.496211,0.345323,0.770224,0.633122,0.661796,0.640584,0.669241,0.676051,0.439864,0.653863,0.380832,0.58462
run2,0.654132,0.758872,0.829522,0.781273,0.733186,0.709402,0.815102,0.671166,0.721847,0.728639,0.772771,0.626025,0.723741,0.784381,0.777436,0.714586
run3,0.823818,0.74214,0.808768,0.776253,0.859104,0.457446,0.847996,0.72558,0.720583,0.801846,0.832654,0.746346,0.792329,0.801922,0.769835,0.576741
run4,0.819251,0.644138,0.834223,0.654975,0.79893,0.430747,0.853952,0.699702,0.791587,0.75893,0.819895,0.717598,0.733721,0.787825,0.69914,0.652736
run5,0.788009,0.671803,0.787947,0.614188,0.488486,0.509963,0.784994,0.761238,0.805165,0.687742,0.708311,0.759981,0.725973,0.758746,0.659486,0.728516
run6,0.790103,0.617743,0.7853,0.590405,0.579693,0.660148,0.762157,0.734771,0.634591,0.634585,0.789858,0.750364,0.729039,0.778429,0.668366,0.767586
run7,0.750286,0.723973,0.799229,0.777541,0.722616,0.519495,0.85875,0.598404,0.7653,0.729525,0.817776,0.758373,0.761399,0.782674,0.746792,0.699633


In [7]:
gen_results2.T.to_csv('result/result_yield_l_rmse.csv', header=False)
gen_results2.T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
source,aats8p,aatsc3d,abcgg,atsc3d,atsc4i,bcutp1l,bertzct,estatevsa3,frnh0,ggi10,kappa2,kappa3,kier2,kier3,peoevsa6,vsaestate3
target,Yield_CO_l,Yield_CO_l,Yield_CO_l,Yield_CO_l,Yield_CO_l,Yield_CO_l,Yield_CO_l,Yield_CO_l,Yield_CO_l,Yield_CO_l,Yield_CO_l,Yield_CO_l,Yield_CO_l,Yield_CO_l,Yield_CO_l,Yield_CO_l
run0,21.580267,22.518127,14.900776,20.118214,20.443417,24.990181,15.332388,22.74124,20.286772,19.283464,19.056622,17.526093,16.670759,16.263401,17.538809,19.7463
run1,17.706652,26.294287,17.764933,26.046593,23.361517,26.631178,15.777181,19.935995,19.14106,19.732189,18.929224,18.733343,24.633366,19.364267,25.898893,21.212881
run2,22.751707,18.996891,15.973221,18.092957,19.983126,20.854761,16.635101,22.184387,20.403297,20.152655,18.441246,23.658104,20.333723,17.963985,18.250996,20.667891
run3,14.807562,17.914078,15.42707,16.687117,13.241938,25.985092,13.754038,18.48036,18.647877,15.703777,14.431482,17.767397,16.076454,15.700772,16.924751,22.951216
run4,15.782238,22.144747,15.114458,21.804966,16.645765,28.008055,14.186598,20.342594,16.947002,18.226454,15.754079,19.727129,19.155745,17.099253,20.361639,21.875593
run5,15.960324,19.858671,15.962644,21.531326,24.792004,24.265945,16.073427,16.938129,15.300875,19.370466,18.721594,16.982656,18.145937,17.026293,20.227884,18.061544
run6,15.871739,21.418999,16.052292,22.1717,22.459742,20.196054,16.89533,17.841536,20.941664,20.941813,15.880988,17.309107,18.033272,16.307135,19.950367,16.701403
run7,18.439034,19.386187,16.533615,17.403717,19.433784,25.57795,13.867908,23.3836,17.876129,19.190228,15.751398,18.137993,18.024086,17.201765,18.567604,20.22287
